# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [2]:
# %pip install pymorphy2
# %pip install razdel
# import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('wordnet')

                                              0.0/55.5 kB ? eta -:--:--
     ------------------------------------     51.2/55.5 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 55.5/55.5 kB 983.5 kB/s eta 0:00:00
                                              0.0/8.2 MB ? eta -:--:--
                                              0.1/8.2 MB 5.5 MB/s eta 0:00:02
     -                                        0.3/8.2 MB 4.3 MB/s eta 0:00:02
     --                                       0.5/8.2 MB 5.6 MB/s eta 0:00:02
     ---                                      0.8/8.2 MB 5.3 MB/s eta 0:00:02
     ----                                     0.9/8.2 MB 4.8 MB/s eta 0:00:02
     ------                                   1.3/8.2 MB 6.1 MB/s eta 0:00:02
     ------                                   1.4/8.2 MB 6.0 MB/s eta 0:00:02
     --------                                 1.8/8.2 MB 6.2 MB/s eta 0:00:02
     ------------                             2.5/8.2 MB 7.4 MB/s eta 0:0

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cvrsd\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cvrsd\AppData\Roaming\nltk_data...
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cvrsd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
from nltk.metrics.distance import edit_distance
from time import perf_counter
import re
import numpy as np

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [2]:
with open("../data sources/litw-win.txt", encoding="windows-1251") as file:
    words = [word.split()[1].strip() for word in file.readlines()]

words[-10:]

['облагодетельствованным',
 'облагодетельствованных',
 'председательствовавший',
 'стадвадцатипятирублевой',
 'высокопревосходительство',
 'высокопревосходительства',
 'попреблагорассмотрительст',
 'попреблагорассмотрительствующемуся',
 'убегающих',
 'уменьшившейся']

In [3]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''.split()

In [15]:
def time_counter(func):
    def wrapper(*args, **kwargs):
        start = perf_counter()
        result = func(*args, **kwargs)
        end = perf_counter() - start
        print(f"Функция отработала за {end} секунды\n")
        
        return result
    return wrapper
        

    

@time_counter
def get_distance(word, bank):
    start = perf_counter()
    lword = len(word)
    sword = set(word)
    fltr = filter(lambda x: (lword - 1 <= len(x) <= lword + 1) and (len(sword - set(x)) <= 2), bank)
    print(f"время на создание списка слов близких по расстоянию Левенштейна для слова - {word:13}: {perf_counter() - start} секунды")
    return fltr

correct_text = [min(get_distance(w, words), key=lambda x: edit_distance(x, w)) if w not in words else w for w in text]


print(f"    Исходный текст - {' '.join(text)}")
print(f"Исправленный текст - {' '.join(correct_text)}")

время на создание списка слов близких по расстоянию Левенштейна для слова - велечайшим   : 3.470000228844583e-05 секунды
Функция отработала за 0.0002106000029016286 секунды

время на создание списка слов близких по расстоянию Левенштейна для слова - Кутузов      : 2.9399991035461426e-05 секунды
Функция отработала за 0.00020119998953305185 секунды

время на создание списка слов близких по расстоянию Левенштейна для слова - уменьшевшейся: 2.7699978090822697e-05 секунды
Функция отработала за 0.0002161000156775117 секунды

    Исходный текст - с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий
Исправленный текст - с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [17]:
raw_text = "Считайте слова из файла litw-win.txt и запишите их в список words. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words."
set_text = set(map(lambda x: x.lower(), re.findall(r"\b[а-яА-ЯеЁ]+\b", raw_text)))

total = sorted(set_text, key=lambda x: re.findall(r"\b[а-яА-ЯеЁ]+\b", raw_text.lower()).index(x))
total

['считайте',
 'слова',
 'из',
 'файла',
 'и',
 'запишите',
 'их',
 'в',
 'список',
 'заданном',
 'предложении',
 'исправьте',
 'все',
 'опечатки',
 'заменив',
 'с',
 'опечатками',
 'на',
 'ближайшие',
 'смысле',
 'расстояния',
 'левенштейна',
 'к',
 'ним',
 'списка',
 'что',
 'слове',
 'есть',
 'опечатка',
 'если',
 'данное',
 'слово',
 'не',
 'содержится',
 'списке']

In [33]:
#%pip install pymorphy3


                                              0.0/55.4 kB ? eta -:--:--
     ------------------------------------     51.2/55.4 kB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 55.4/55.4 kB 1.4 MB/s eta 0:00:00
                                              0.0/8.4 MB ? eta -:--:--
                                              0.1/8.4 MB 3.3 MB/s eta 0:00:03
     -                                        0.3/8.4 MB 5.1 MB/s eta 0:00:02
     --                                       0.6/8.4 MB 5.4 MB/s eta 0:00:02
     ---                                      0.7/8.4 MB 5.7 MB/s eta 0:00:02
     ----                                     1.0/8.4 MB 5.4 MB/s eta 0:00:02
     ------                                   1.4/8.4 MB 6.9 MB/s eta 0:00:02
     -------                                  1.6/8.4 MB 6.6 MB/s eta 0:00:02
     -----------                              2.5/8.4 MB 8.3 MB/s eta 0:00:01
     -------------                            2.9/8.4 MB 8.3 MB/s eta 0:0

In [18]:
import nltk
from nltk.stem import SnowballStemmer
import pandas as pd
import pymorphy3


stemmer = SnowballStemmer('russian')
morph = pymorphy3.MorphAnalyzer()


lemmes = [morph.parse(word)[0].normal_form for word in total]
stemmes = [stemmer.stem(word) for word in total]

dframe = pd.DataFrame({
    'word': total,
    'lemme': lemmes,
    'stemm': stemmes
})

dframe

,word,lemme,stemm
0,считайте,считать,счита
1,слова,слово,слов
2,из,из,из
3,файла,файл,файл
4,и,и,и
5,запишите,записать,запиш
6,их,они,их
7,в,в,в
8,список,список,список
9,заданном,задать,зада


In [ ]:
"""
def get_info(word):
    return f"Слово - {word}\nЛемматизация - {morph.parse(word)[0].normal_form}\nСтемминг - {stemmer.stem(word)}\n{'-' * 25}"


#[print(get_info(word)) for word in ltext]
for word in ltext:
    print(get_info(word))
"""

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [19]:
from razdel import sentenize

count_vectorizer = CountVectorizer()
snt = [list(row)[-1] for row in sentenize(raw_text)]

In [20]:
wcv = count_vectorizer.fit_transform(snt)
wcv.toarray()

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]], dtype=int64)

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [21]:
from nltk import word_tokenize

# Из-за большого веса (500 Мб) preprocessed_descriptions.csv не смог загрузить его на гит, поэтому разбил его на чанки по 23 Мб
'''
descriptions = pd.read_parquet("../data sources/preprocessed_descriptions chunks/chunk1.parquet")

for i in range(2, 11):
    df = pd.read_parquet(f"../data sources/preprocessed_descriptions chunks/chunk{i}.parquet")
    bank = pd.concat([bank, df], ignore_index=True)

descriptions.size
'''


descriptions = pd.read_csv('../data sources/preprocessed_descriptions.csv')

descriptions.size

2231637

In [24]:
#ограничил вывод, т.к. слишком долгое ожидание

d_list = descriptions.descriptions.to_list()[:1000] #Взял первую тысячу описаний
raw_words = [word_tokenize(row) for row in d_list]
clear_words = [[word for word in row if word.isalpha()] for row in raw_words]
unique_words = [sorted(set(row), key=lambda x: row.index(x)) for row in clear_words]



In [25]:
print(f"Из описания:\n\n{d_list[0]}\n\nУникальные слова:\n{unique_words[0]} ")

Из описания:

remove the fat cap and any large areas or pockets of external fat that can be easily trimmed away cut the eggplants in half lengthwise and  taking care not to break the skins  gently scoop out the flesh bake uncovered in a 350f for one hour or until golden and hot through refrigerate

Уникальные слова:
['remove', 'the', 'fat', 'cap', 'and', 'any', 'large', 'areas', 'or', 'pockets', 'of', 'external', 'that', 'can', 'be', 'easily', 'trimmed', 'away', 'cut', 'eggplants', 'in', 'half', 'lengthwise', 'taking', 'care', 'not', 'to', 'break', 'skins', 'gently', 'scoop', 'out', 'flesh', 'bake', 'uncovered', 'a', 'for', 'one', 'hour', 'until', 'golden', 'hot', 'through', 'refrigerate'] 


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [48]:
f = lambda: np.random.randint(0, len(unique_words))
five_pairs = [(unique_words[f()][0], unique_words[f()][0]) for i in range(5)]
five_pairs

[('cut', 'tear'),
 ('combine', 'combine'),
 ('brown', 'cut'),
 ('boil', 'preheat'),
 ('heat', 'stir')]

In [49]:
[print(f"{'Расстояние редактирования между словами:':>30} {v1:^7} и {v2:^7} - {edit_distance(v1, v2):<20}") for v1, v2 in five_pairs]

Расстояние редактирования между словами:   cut   и  tear   - 4                   
Расстояние редактирования между словами: combine и combine - 0                   
Расстояние редактирования между словами:  brown  и   cut   - 5                   
Расстояние редактирования между словами:  boil   и preheat - 7                   
Расстояние редактирования между словами:  heat   и  stir   - 4                   


[None, None, None, None, None]

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [50]:
words[-10:]

['облагодетельствованным',
 'облагодетельствованных',
 'председательствовавший',
 'стадвадцатипятирублевой',
 'высокопревосходительство',
 'высокопревосходительства',
 'попреблагорассмотрительст',
 'попреблагорассмотрительствующемуся',
 'убегающих',
 'уменьшившейся']

In [68]:
def get_similar(word, k, words):
    
    fltr = list(get_distance(word, words))
    print(f"Отфильтровал слова, оставил наиболее похожие, которые прошли фильтр:\n{fltr}\n\n")
    
    result = sorted(fltr, key=lambda x: edit_distance(word, x))[:k]
    print(f"Наиболее близкие по расстоянию Левенштейна слова к слову '{word}':\nКоличество - {k}:\n{result}")
    return result
    

get_similar("убегающий", 5, words)

время на создание списка слов близких по расстоянию Левенштейна для слова - убегающий    : 1.300001167692244e-05 секунды
Функция отработала за 0.0005393000028561801 секунды

Отфильтровал слова, оставил наиболее похожие, которые прошли фильтр:
['пугающий', 'балующий', 'бегающих', 'требующий', 'убивающий', 'умирающей', 'угасающий', 'бегающими', 'угасающие', 'играющему', 'огибающей', 'огибающую', 'сбегающей', 'слабеющий', 'убегающая', 'убегающей', 'убегающим', 'убивающие', 'уезжающий', 'улыбающег', 'умирающего', 'богатейшую', 'угрожающей', 'блуждающей', 'блуждающий', 'погибающий', 'блистающей', 'забегающие', 'набегающий', 'небрегущий', 'негибнущая', 'обитающего', 'обрекающий', 'оглушающий', 'разумеющий', 'убегающего', 'угнетающим', 'угрожающий', 'упрекающий', 'убегающих']


Наиболее близкие по расстоянию Левенштейна слова к слову 'убегающий':
Количество - 5:
['убегающей', 'убегающим', 'убегающих', 'бегающих', 'убивающий']


['убегающей', 'убегающим', 'убегающих', 'бегающих', 'убивающий']

In [78]:
get_similar("Сабиров", 7, words)

время на создание списка слов близких по расстоянию Левенштейна для слова - Сабиров      : 1.2500007869675756e-05 секунды
Функция отработала за 0.0007728000055067241 секунды

Отфильтровал слова, оставил наиболее похожие, которые прошли фильтр:
['бросив', 'обрати', 'прибав', 'выбора', 'обрыва', 'собрав', 'товари', 'грибов', 'вообра', 'бориса', 'браков', 'бритва', 'изобра', 'обвила', 'разбои', 'борова', 'бровки', 'ворами', 'избрав', 'изорва', 'мирабо', 'обирая', 'собира', 'авирон', 'аридов', 'ариево', 'барино', 'биогра', 'бисова', 'бравое', 'бритво', 'бровка', 'вбирал', 'выбира', 'вырабо', 'говари', 'добива', 'добира', 'крабов', 'обивая', 'обрами', 'овраги', 'оправи', 'отрави', 'пирова', 'пробив', 'провиа', 'рабочи', 'разбив', 'кровати', 'бросила', 'товарищ', 'обратил', 'арбузов', 'красиво', 'барином', 'бровями', 'добавил', 'оборвал', 'правило', 'бросали', 'выбрали', 'образов', 'ограбил', 'рабочих', 'рабство', 'баранов', 'выбирая', 'обратив', 'рабочие', 'собирал', 'собрали', 'чебаров', '

['заборов', 'авирон', 'аридов', 'чебаров', 'базаров', 'амбаров', 'париков']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [50]:
from nltk.stem import WordNetLemmatizer

#nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cvrsd\AppData\Roaming\nltk_data...


True

In [51]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('russian')

word = [item for sublist in unique_words for item in sublist]
stemme = [stemmer.stem(w) for w in word]
lemme = [lemmatizer.lemmatize(w, pos='v') for w in word]

df = pd.DataFrame({
    "word": word,
    "stemmed_word": stemme,
    "normalized_word": lemme
})
df = df.set_index("word")
df

,stemmed_word,normalized_word
word,,
remove,remove,remove
the,the,the
fat,fat,fat
cap,cap,cap
and,and,and
...,...,...
cassarole,cassarole,cassarole
dish,dish,dish
bake,bake,bake


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [81]:
from collections import Counter
from nltk.corpus import stopwords

sw = stopwords.words("english")

In [53]:

before = Counter(word)
after = Counter([w for w in word if w not in sw])

print(f"Доля которую составляли стоп-слова: {round((len(before) - len(after)) / len(before), 2) * 100}%")
len(before) - len(after), len(before)

Доля которую составляли стоп-слова: 4.0%


(96, 2343)

In [230]:
most_popular_before = sorted(before.items(), key=lambda x: x[1], reverse=True)[:10]
most_popular_after = sorted(after.items(), key=lambda x: x[1], reverse=True)[:10]
print(f"Самые популярные слова\n{'-'*100}\n|{'до:':^49}|{'после:':^48}|\n{'-'*100}")
[print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|\n{'-'*100}") for v1, v2 in zip(most_popular_before, most_popular_after)]

Самые популярные слова
----------------------------------------------------------------------------------------------------
|                       до:                       |                     после:                     |
----------------------------------------------------------------------------------------------------
|           and                     796           |          add                     415           |
----------------------------------------------------------------------------------------------------
|           in                      692           |          heat                    315           |
----------------------------------------------------------------------------------------------------
|            a                      651           |        minutes                   289           |
----------------------------------------------------------------------------------------------------
|           the                     646           |          bowl   

[None, None, None, None, None, None, None, None, None, None]

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [75]:
#d_list[:5] # список описаний

# vectorizer = TfidfVectorizer()

# raw_recipes5 = [d_list[np.random.randint(0, len(d_list))] for i in range(5)]

# raw_recipes5 # список 5 сырых рандомно выбранных описаний


# clear_recipes5 = [[word for word in word_tokenize(row) if word.isalpha()] for row in raw_recipes5]

# clear_recipes5 # список токенизированных описаний.

# from pprint import pprint

# matrix = vectorizer.fit_transform([' '.join(recipe) for recipe in clear_recipes5])

# pprint(matrix.toarray())

In [87]:
data = descriptions.sample(5)
data

,descriptions
951530,for the cake: put the basil vinegar water and...
1457714,heat oven to 325 degrees prepare cruets of dre...
1574640,bring a large pot of lightly salted water to a...
630893,mix well all ingredients in a blender brown po...
176000,preheat oven to 350 degrees


In [88]:
vectorizer = TfidfVectorizer(analyzer="word", stop_words="english")

vectorizer.fit(data["descriptions"])

def vectorizer_processing(x):
    sents = [x["descriptions"]]
    vector = vectorizer.transform(sents)
    return vector.toarray()

data['TfidfVectorizer'] = data.apply(lambda x: vectorizer_processing(x), axis=1)


values = [(k,v) for (k, v) in dict(vectorizer.vocabulary_).items()]
values.sort(key=lambda x: x[1])
values

[('10', 0),
 ('11x14', 1),
 ('25', 2),
 ('325', 3),
 ('35', 4),
 ('350', 5),
 ('375', 6),
 ('add', 7),
 ('al', 8),
 ('aside', 9),
 ('bag', 10),
 ('bake', 11),
 ('baking', 12),
 ('basil', 13),
 ('bell', 14),
 ('blender', 15),
 ('boil', 16),
 ('bring', 17),
 ('broth', 18),
 ('brown', 19),
 ('butter', 20),
 ('cake', 21),
 ('casserole', 22),
 ('cheddar', 23),
 ('cheese', 24),
 ('chicken', 25),
 ('cloves', 26),
 ('coat', 27),
 ('cook', 28),
 ('cream', 29),
 ('creamy', 30),
 ('cruets', 31),
 ('degrees', 32),
 ('dente', 33),
 ('dish', 34),
 ('drain', 35),
 ('dressing', 36),
 ('frequently', 37),
 ('garlic', 38),
 ('greased', 39),
 ('grits', 40),
 ('heat', 41),
 ('heated', 42),
 ('high', 43),
 ('inch', 44),
 ('ingredients', 45),
 ('large', 46),
 ('let', 47),
 ('lightly', 48),
 ('low', 49),
 ('mash', 50),
 ('medium', 51),
 ('melt', 52),
 ('minute', 53),
 ('minutes', 54),
 ('mix', 55),
 ('mixture', 56),
 ('mushroom', 57),
 ('mushrooms', 58),
 ('oil', 59),
 ('onion', 60),
 ('onions', 61),
 ('oven'

In [116]:
for word, vector in zip(data["descriptions"].to_list(), data["TfidfVectorizer"].to_list()):
    print(vector.shape)
    print(f"{word}\n\n{vector}\n{'-'*vector.shape[-1]}\n")

(1, 101)
for the cake: put the basil vinegar  water and salt in a saucepan over high heat

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.39935681 0.         0.         0.         0.
  0.         0.         0.         0.39935681 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.22499066
  0.         0.39935681 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.32219856
  0.         0.         0.32219856 0.         0.         0.
  0.         0.         0.         0.         0.     

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние `(scipy.spatial.distance.cosine)` Результаты оформите в виде таблицы pd.DataFrame. В качестве названий строк и столбцов используйте названия рецептов.

In [90]:
data

,descriptions,TfidfVectorizer
951530,for the cake: put the basil vinegar water and...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1457714,heat oven to 325 degrees prepare cruets of dre...,"[[0.1690745368712321, 0.0, 0.0, 0.209563530479..."
1574640,bring a large pot of lightly salted water to a...,"[[0.08553215520668865, 0.10601490174862549, 0...."
630893,mix well all ingredients in a blender brown po...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
176000,preheat oven to 350 degrees,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.6264769620593381,..."


In [152]:
from scipy.spatial.distance import cosine
from itertools import product

df = pd.DataFrame(columns=data["descriptions"].values, index=data["descriptions"].values)

df

,for the cake: put the basil vinegar water and salt in a saucepan over high heat,heat oven to 325 degrees prepare cruets of dressing reduce heat to low and simmer stirring frequently until grits are creamy - about 10 minute place 1 / 2 teaspoon of oil and seeds into a large ziplock bag and shake to coat,bring a large pot of lightly salted water to a boil add pasta and cook for 8 to 10 minutes or until al dente drain and set aside preheat oven to 375 degrees f melt butter in a large saucepan over medium heat add mushrooms onion and bell pepper and saute until tender stir in cream of mushroom soup chicken broth and cook stirring until heated through stir in pasta cheddar cheese peas sherry worcestershire sauce salt pepper and chicken mix well and transfer mixture to a lightly greased 11x14 inch baking dish sprinkle with parmesan cheese and paprika bake in the preheated oven for 25 to 35 minutes or until heated through,mix well all ingredients in a blender brown pork all over in oil in casserole dish in a skillet heat the oil over medium heat and saute the onions until they are transluscent mash the garlic cloves into a paste and set aside let stand overnight,preheat oven to 350 degrees
for the cake: put the basil vinegar water and salt in a saucepan over high heat,NaN,NaN,NaN,NaN,NaN
heat oven to 325 degrees prepare cruets of dressing reduce heat to low and simmer stirring frequently until grits are creamy - about 10 minute place 1 / 2 teaspoon of oil and seeds into a large ziplock bag and shake to coat,NaN,NaN,NaN,NaN,NaN
bring a large pot of lightly salted water to a boil add pasta and cook for 8 to 10 minutes or until al dente drain and set aside preheat oven to 375 degrees f melt butter in a large saucepan over medium heat add mushrooms onion and bell pepper and saute until tender stir in cream of mushroom soup chicken broth and cook stirring until heated through stir in pasta cheddar cheese peas sherry worcestershire sauce salt pepper and chicken mix well and transfer mixture to a lightly greased 11x14 inch baking dish sprinkle with parmesan cheese and paprika bake in the preheated oven for 25 to 35 minutes or until heated through,NaN,NaN,NaN,NaN,NaN
mix well all ingredients in a blender brown pork all over in oil in casserole dish in a skillet heat the oil over medium heat and saute the onions until they are transluscent mash the garlic cloves into a paste and set aside let stand overnight,NaN,NaN,NaN,NaN,NaN
preheat oven to 350 degrees,NaN,NaN,NaN,NaN,NaN


In [156]:
pairs = list(product(data["TfidfVectorizer"].values, repeat=2)) 

bank = [cosine(p1.ravel(), p2.ravel()) for p1, p2 in pairs]

arr = np.array(bank).reshape((5, 5))
arr

array([[0.        , 0.94687321, 0.90388698, 0.94688308, 1.        ],
       [0.94687321, 0.        , 0.89815781, 0.88709175, 0.88223221],
       [0.90388698, 0.89815781, 0.        , 0.89914764, 0.86740351],
       [0.94688308, 0.88709175, 0.89914764, 0.        , 1.        ],
       [1.        , 0.88223221, 0.86740351, 1.        , 0.        ]])

In [157]:
df[:] = arr
df

,for the cake: put the basil vinegar water and salt in a saucepan over high heat,heat oven to 325 degrees prepare cruets of dressing reduce heat to low and simmer stirring frequently until grits are creamy - about 10 minute place 1 / 2 teaspoon of oil and seeds into a large ziplock bag and shake to coat,bring a large pot of lightly salted water to a boil add pasta and cook for 8 to 10 minutes or until al dente drain and set aside preheat oven to 375 degrees f melt butter in a large saucepan over medium heat add mushrooms onion and bell pepper and saute until tender stir in cream of mushroom soup chicken broth and cook stirring until heated through stir in pasta cheddar cheese peas sherry worcestershire sauce salt pepper and chicken mix well and transfer mixture to a lightly greased 11x14 inch baking dish sprinkle with parmesan cheese and paprika bake in the preheated oven for 25 to 35 minutes or until heated through,mix well all ingredients in a blender brown pork all over in oil in casserole dish in a skillet heat the oil over medium heat and saute the onions until they are transluscent mash the garlic cloves into a paste and set aside let stand overnight,preheat oven to 350 degrees
for the cake: put the basil vinegar water and salt in a saucepan over high heat,0.0,0.946873,0.903887,0.946883,1.0
heat oven to 325 degrees prepare cruets of dressing reduce heat to low and simmer stirring frequently until grits are creamy - about 10 minute place 1 / 2 teaspoon of oil and seeds into a large ziplock bag and shake to coat,0.946873,0.0,0.898158,0.887092,0.882232
bring a large pot of lightly salted water to a boil add pasta and cook for 8 to 10 minutes or until al dente drain and set aside preheat oven to 375 degrees f melt butter in a large saucepan over medium heat add mushrooms onion and bell pepper and saute until tender stir in cream of mushroom soup chicken broth and cook stirring until heated through stir in pasta cheddar cheese peas sherry worcestershire sauce salt pepper and chicken mix well and transfer mixture to a lightly greased 11x14 inch baking dish sprinkle with parmesan cheese and paprika bake in the preheated oven for 25 to 35 minutes or until heated through,0.903887,0.898158,0.0,0.899148,0.867404
mix well all ingredients in a blender brown pork all over in oil in casserole dish in a skillet heat the oil over medium heat and saute the onions until they are transluscent mash the garlic cloves into a paste and set aside let stand overnight,0.946883,0.887092,0.899148,0.0,1.0
preheat oven to 350 degrees,1.0,0.882232,0.867404,1.0,0.0


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

`Минимально похожими являются те описания у которых коэффициент равен или ближе всего к 1`
